In [1]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from keras.models import Sequential
from keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-01-20 16:03:07.409419: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
TIMEFRAMES = [7, 14, 30, 90, 180]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['MPA', 'CPA', 'MLC', 'API']

labels = pd.read_csv('labels/labels.csv')

In [ ]:
# Find bounding boxes of all 4 image sizes
def find_bounding_box(img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find all non-black pixels
    coords = cv2.findNonZero(gray)
    
    # Find the bounding box of those pixels
    x, y, w, h = cv2.boundingRect(coords)
    print(x, y, w, h)

# Define the paths to your images
image_paths = [
    'images/MLC/AI&Robotics_2019-02-01 00:00:00_7.png',
    'images/MLC/AI&Robotics_2019-02-01 00:00:00_14.png',
    'images/MLC/AI&Robotics_2019-02-01 00:00:00_30.png',
    'images/MLC/AI&Robotics_2019-02-01 00:00:00_90.png',
    'images/MLC/AI&Robotics_2019-02-01 00:00:00_180.png',   
]

# Crop each image and save the result
for path in image_paths:
    cropped_image = find_bounding_box(path)

In [3]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 7:
            # Crop the image using the bounding rectangle
            crop = img[105:105+115, 80:80+38]
        elif number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [4]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_8281/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [5]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [6]:
# Sort the DataFrame by date
labels['Date'] = labels['Image'].str.extract(r'(\d{4}-\d{2}-\d{2})')
labels['Date'] = pd.to_datetime(labels['Date'])
labels = labels.sort_values(by='Date') 
print(labels.head())

                                                  Image  TimePrediction  \
0     images/MPA/S&P500_2015-01-02 00:00:00_7_croppe...               5   
1328  images/MPA/S&P500_2015-01-02 00:00:00_180_crop...               5   
1329  images/CPA/S&P500_2015-01-02 00:00:00_180_crop...               5   
1331  images/API/S&P500_2015-01-02 00:00:00_180_crop...               5   
1332  images/MPA/S&P500_2015-01-02 00:00:00_180_crop...              30   

      LastPrice  FuturePrice  Label  \
0        204.25   200.860001      0   
1328     207.50   206.720001      0   
1329     207.50   206.720001      0   
1331     207.50   206.720001      0   
1332     207.50   210.500000      1   

                                            Image_Array       Date  
0     [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2015-01-02  
1328  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2015-01-02  
1329  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,... 2015-01-02  
1331  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [7]:
# Show the number of 1s and 0s in the dataset
print(labels['Label'].value_counts())

1    5132
0    4208
Name: Label, dtype: int64


In [8]:
def create_cnn_model(num_blocks, initial_filters=64, input_shape=()):
    # Determine the number of blocks based on the input image size
    if num_blocks == 2:
        dilation_rate = (1, 1)
    elif num_blocks == 3:
        dilation_rate = (1, 2)
    else:
        dilation_rate = (1, 3)

    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Add Convolutional layer
        if i == 0:  # Apply specific strides and dilation rate only for the first layer
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 3), padding='same', 
                             input_shape=input_shape if i == 0 else None))
            model.add(Conv2D(filters, kernel_size=(5, 3), strides=(1, 1), dilation_rate=dilation_rate, padding='same', 
                             input_shape=input_shape if i == 0 else None))
        else:
            model.add(Conv2D(filters, kernel_size=(5, 3), padding='same'))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer
        model.add(MaxPooling2D(pool_size=(2, 1)))

    # Add Flatten and Dense layers for final prediction
    #dense_units = {2: 64, 3: 128, 4: 264}.get(num_blocks, 64) 
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss=focal_loss(gamma=2., alpha=0.25), metrics=['accuracy'])

    return model

In [9]:
def get_investment_return(y_pred_binary, lastPrice, futurePrice):
    # Calculate Rate of Return RoR for each trade independently
    # Long positions
    long_ror = []
    long_trades = []
    long_investment = 0
    # Short positions
    short_ror = []
    short_trades = []
    short_investment = 0

    y_pred_binary = y_pred_binary.tolist()
    for i in range(len(y_pred_binary)):
        # Long position
        if y_pred_binary[i][0] == 1:
            long_return_i = ((futurePrice[i] - lastPrice[i])/lastPrice[i])
            long_ror.append(long_return_i)
            long_trades.append(100*(long_return_i))
            long_investment += 100
        # Short position
        else:
            short_return_i = ((lastPrice[i] - futurePrice[i])/lastPrice[i])
            short_ror.append(short_return_i)
            short_trades.append(100*(short_return_i))
            short_investment += 100
    
    if long_investment > 0:
        # Calculate average long RoR
        long_avg_ror = np.mean(long_ror)
        # Sum all long trades
        long_trades = np.sum(long_trades)
    else:
        long_avg_ror = 0
        long_trades = 0
    
    if short_investment > 0:
        # Calculate average short RoR
        short_avg_ror = np.mean(short_ror)
        # Sum all short trades
        short_trades = np.sum(short_trades)
    else:
        short_avg_ror = 0
        short_trades = 0

    return long_avg_ror, long_trades, long_investment, short_avg_ror, short_trades, short_investment

In [ ]:
# Train and evaluate the model for each img_type, timeframe, and prediction

evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction <= timeframe:
                print(f"Predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")
                
                # Initialize variables for weighted sums and total count
                weighted_accuracy_sum = 0
                weighted_precision_sum = 0
                weighted_recall_sum = 0
                weighted_f1_score_sum = 0
                weighted_hit_rate_sum = 0
                total_predictions = 0
                long_avg_ror = []
                long_trades = []
                long_investment = []
                short_avg_ror = []
                short_trades = []
                short_investment = []
                total_investment = []
                total_returns = []
                
                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) &
                              (labels['Image'].str.contains(f'/{img_type}/')) &
                              (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
                
                # Create and compile your CNN model based on timeframe
                if timeframe == 7:
                    model = create_cnn_model(2, input_shape=(115, 38, 1))
                elif timeframe == 14:
                    model = create_cnn_model(2, input_shape=(120, 85, 1))
                elif timeframe == 30:
                    model = create_cnn_model(3, input_shape=(120, 132, 1))
                elif timeframe == 90:
                    model = create_cnn_model(3, input_shape=(120, 226, 1))
                elif timeframe == 180:
                    model = create_cnn_model(4, input_shape=(120, 414, 1))
                
                X = np.array(data['Image_Array'].tolist()) / 255.0
                lastPrice = data['LastPrice'].tolist()
                futurePrice = data['FuturePrice'].tolist()
                y = data['Label'].values

                # Define the rolling window size for training within the initial training set
                window_size = len(X) / 10

                # Early stopping callback
                early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

                for window in range(5):
                    start = 0
                    if window == 0:
                        end = int(len(X) * 0.4)
                    else:
                        start = end
                        end = start + int(window_size)

                    # Buffer = window size
                    # Define end of buffer
                    buffer_end = end + int(window_size)

                    # Ensure buffer_end does not exceed the length of the initial training set
                    buffer_end = min(buffer_end, len(X))

                    # Split the data into training and test sets in temporal order
                    X_train_temp = X[start:end]
                    y_train_temp = y[start:end]
                    lastPrice_train_temp = lastPrice[start:end]
                    futurePrice_train_temp = futurePrice[start:end]

                    X_test_temp = X[buffer_end:] if window == 4 else X[buffer_end:buffer_end+int(window_size*2)]
                    y_test_temp = y[buffer_end:] if window == 4 else y[buffer_end:buffer_end+int(window_size*2)]
                    lastPrice_test_temp = lastPrice[buffer_end:] if window == 4 else lastPrice[buffer_end:buffer_end+int(window_size*2)]
                    futurePrice_test_temp = futurePrice[buffer_end:] if window == 4 else futurePrice[buffer_end:buffer_end+int(window_size*2)]

                    #model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=10, validation_split=0.2, class_weight=class_weight_dict) 
                    model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=10, validation_split=0.2)  
                
                    # Evaluate the model
                    y_pred = model.predict(X_test_temp)
                    # Convert predictions to binary: if > 0.5 then 1 else 0
                    y_pred_binary = np.where(y_pred > 0.5, 1, 0)

                    num_predictions = len(y_test_temp)
                    
                    # Calculate the performance metrics for the current epoch
                    current_accuracy = accuracy_score(y_test_temp, y_pred_binary)
                    current_precision = precision_score(y_test_temp, y_pred_binary)
                    current_recall = recall_score(y_test_temp, y_pred_binary)
                    current_f1_score = fbeta_score(y_test_temp, y_pred_binary, beta=1)

                    y_test_array = y_test_temp.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                    correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                    current_hit_rate = (correct_predictions / num_predictions)

                    # Update the weighted sums and total count
                    weighted_accuracy_sum += current_accuracy * num_predictions
                    weighted_precision_sum += current_precision * num_predictions
                    weighted_recall_sum += current_recall * num_predictions
                    weighted_f1_score_sum += current_f1_score * num_predictions
                    weighted_hit_rate_sum += current_hit_rate * num_predictions
                    total_predictions += num_predictions

                    long_avg_ror_temp, long_trades_temp, long_investment_temp, short_avg_ror_temp, short_trades_temp, short_investment_temp = get_investment_return(y_pred_binary, lastPrice_test_temp, futurePrice_test_temp)
                    # Total investment
                    long_avg_ror.append(long_avg_ror_temp)
                    long_trades.append(long_trades_temp)
                    long_investment.append(long_investment_temp)
                    short_avg_ror.append(short_avg_ror_temp)
                    short_trades.append(short_trades_temp)
                    short_investment.append(short_investment_temp)
                
                # replace nan values with 0
                long_avg_ror = [0 if np.isnan(x) else x for x in long_avg_ror]
                long_trades = [0 if np.isnan(x) else x for x in long_trades]
                long_investment = [0 if np.isnan(x) else x for x in long_investment]
                short_avg_ror = [0 if np.isnan(x) else x for x in short_avg_ror]
                short_trades = [0 if np.isnan(x) else x for x in short_trades]
                short_investment = [0 if np.isnan(x) else x for x in short_investment]

                # Calculate the weighted averages after the epoch loop
                weighted_average_accuracy = weighted_accuracy_sum / total_predictions
                weighted_average_precision = weighted_precision_sum / total_predictions
                weighted_average_recall = weighted_recall_sum / total_predictions
                weighted_average_f1_score = weighted_f1_score_sum / total_predictions
                weighted_average_hit_rate = weighted_hit_rate_sum / total_predictions
                            
                print("Evaluation Metrics:")
                print(f"Accuracy: {weighted_average_accuracy}")
                print(f"Precision: {weighted_average_precision}")
                print(f"Recall: {weighted_average_recall}")
                print(f"F1 Score: {weighted_average_f1_score}")
                print(f"Hit Rate: {weighted_average_hit_rate}")
                print(f"Average Long RoR: {np.mean(long_avg_ror)}")
                print(f"Long Investment: {np.mean(long_investment)}")
                print(f"Long Return: {np.mean(long_trades)}")
                print(f"Average Short RoR: {np.mean(short_avg_ror)}")
                print(f"Short Investment: {np.mean(short_investment)}")
                print(f"Short Return: {np.mean(short_trades)}")

                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': weighted_average_accuracy,
                    'Precision': weighted_average_precision,
                    'Recall': weighted_average_recall,
                    'F1_Score': weighted_average_f1_score,
                    'Hit_Rate': weighted_average_hit_rate,
                    'Long_Average_RoR': np.mean(long_avg_ror),
                    'Long_Investment': np.mean(long_investment),
                    'Long_Return': np.mean(long_trades),
                    'Short_Average_RoR': np.mean(short_avg_ror),
                    'Short_Investment': np.mean(short_investment),
                    'Short_Return': np.mean(short_trades)
                }, ignore_index=True)
                
                # Save the model
                model.save(f"models/noWeights_{img_type}_{timeframe}_{prediction}.h5")
                print(f"Model saved as {img_type}_{timeframe}_{prediction}.h5")
                print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation/separate/evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'evaluation/separate/evaluation_scores.csv'.")

In [10]:
# Train models on all images types and evaluate them based on timeframe and prediction

evaluation_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F1_Score', 'Hit_Rate', 'Long_Average_RoR', 'Long_Investment', 'Long_Return', 'Short_Average_RoR', 'Short_Investment', 'Short_Return'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction <= timeframe:
            print(f"Predicting {prediction} days ahead with {timeframe} days timeframe.")

            # Initialize variables for weighted sums and total count
            weighted_accuracy_sum = 0
            weighted_precision_sum = 0
            weighted_recall_sum = 0
            weighted_f1_score_sum = 0
            weighted_hit_rate_sum = 0
            total_predictions = 0
            long_avg_ror = []
            long_trades = []
            long_investment = []
            short_avg_ror = []
            short_trades = []
            short_investment = []
            total_investment = []
            total_returns = []
            
            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
            
            # Create and compile your CNN model based on timeframe
            if timeframe == 7:
                model = create_cnn_model(2, input_shape=(115, 38, 1))
            elif timeframe == 14:
                model = create_cnn_model(2, input_shape=(120, 85, 1))
            elif timeframe == 30:
                model = create_cnn_model(3, input_shape=(120, 132, 1))
            elif timeframe == 90:
                model = create_cnn_model(3, input_shape=(120, 226, 1))
            elif timeframe == 180:
                model = create_cnn_model(4, input_shape=(120, 414, 1))
            
            X = np.array(data['Image_Array'].tolist()) / 255.0
            lastPrice = data['LastPrice'].tolist()
            futurePrice = data['FuturePrice'].tolist()
            y = data['Label'].values

            # Define the rolling window size for training within the initial training set
            window_size = len(X) / 10

            # Early stopping callback
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

            for window in range(5):
                start = 0
                if window == 0:
                    end = int(len(X) * 0.4)
                else:
                    start = end
                    end = start + int(window_size)

                # Buffer = window size
                # Define end of buffer
                buffer_end = end + int(window_size)

                # Ensure buffer_end does not exceed the length of the initial training set
                buffer_end = min(buffer_end, len(X))

                # Split the data into training and test sets in temporal order
                X_train_temp = X[start:end]
                y_train_temp = y[start:end]
                lastPrice_train_temp = lastPrice[start:end]
                futurePrice_train_temp = futurePrice[start:end]

                X_test_temp = X[buffer_end:] if window == 4 else X[buffer_end:buffer_end+int(window_size*2)]
                y_test_temp = y[buffer_end:] if window == 4 else y[buffer_end:buffer_end+int(window_size*2)]
                lastPrice_test_temp = lastPrice[buffer_end:] if window == 4 else lastPrice[buffer_end:buffer_end+int(window_size*2)]
                futurePrice_test_temp = futurePrice[buffer_end:] if window == 4 else futurePrice[buffer_end:buffer_end+int(window_size*2)]

                #model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=10, validation_split=0.2, class_weight=class_weight_dict) 
                model.fit(X_train_temp, y_train_temp, batch_size=32, epochs=10, validation_split=0.2)  
            
                # Evaluate the model
                y_pred = model.predict(X_test_temp)
                # Convert predictions to binary: if > 0.5 then 1 else 0
                y_pred_binary = np.where(y_pred > 0.5, 1, 0)

                num_predictions = len(y_test_temp)
                
                # Calculate the performance metrics for the current epoch
                current_accuracy = accuracy_score(y_test_temp, y_pred_binary)
                current_precision = precision_score(y_test_temp, y_pred_binary)
                current_recall = recall_score(y_test_temp, y_pred_binary)
                current_f1_score = fbeta_score(y_test_temp, y_pred_binary, beta=1)

                y_test_array = y_test_temp.ravel()  # Convert y_test to a 1D NumPy array if it's a pandas Series
                correct_predictions = np.sum(y_pred_binary.ravel() == y_test_array)
                current_hit_rate = (correct_predictions / num_predictions)

                # Update the weighted sums and total count
                weighted_accuracy_sum += current_accuracy * num_predictions
                weighted_precision_sum += current_precision * num_predictions
                weighted_recall_sum += current_recall * num_predictions
                weighted_f1_score_sum += current_f1_score * num_predictions
                weighted_hit_rate_sum += current_hit_rate * num_predictions
                total_predictions += num_predictions

                long_avg_ror_temp, long_trades_temp, long_investment_temp, short_avg_ror_temp, short_trades_temp, short_investment_temp = get_investment_return(y_pred_binary, lastPrice_test_temp, futurePrice_test_temp)
                # Total investment
                long_avg_ror.append(long_avg_ror_temp)
                long_trades.append(long_trades_temp)
                long_investment.append(long_investment_temp)
                short_avg_ror.append(short_avg_ror_temp)
                short_trades.append(short_trades_temp)
                short_investment.append(short_investment_temp)
            
            # replace nan values with 0
            long_avg_ror = [0 if np.isnan(x) else x for x in long_avg_ror]
            long_trades = [0 if np.isnan(x) else x for x in long_trades]
            long_investment = [0 if np.isnan(x) else x for x in long_investment]
            short_avg_ror = [0 if np.isnan(x) else x for x in short_avg_ror]
            short_trades = [0 if np.isnan(x) else x for x in short_trades]
            short_investment = [0 if np.isnan(x) else x for x in short_investment]

            # Calculate the weighted averages after the epoch loop
            weighted_average_accuracy = weighted_accuracy_sum / total_predictions
            weighted_average_precision = weighted_precision_sum / total_predictions
            weighted_average_recall = weighted_recall_sum / total_predictions
            weighted_average_f1_score = weighted_f1_score_sum / total_predictions
            weighted_average_hit_rate = weighted_hit_rate_sum / total_predictions
                        
            print("Evaluation Metrics:")
            print(f"Accuracy: {weighted_average_accuracy}")
            print(f"Precision: {weighted_average_precision}")
            print(f"Recall: {weighted_average_recall}")
            print(f"F1 Score: {weighted_average_f1_score}")
            print(f"Hit Rate: {weighted_average_hit_rate}")
            print(f"Average Long RoR: {np.mean(long_avg_ror)}")
            print(f"Long Investment: {np.mean(long_investment)}")
            print(f"Long Return: {np.mean(long_trades)}")
            print(f"Average Short RoR: {np.mean(short_avg_ror)}")
            print(f"Short Investment: {np.mean(short_investment)}")
            print(f"Short Return: {np.mean(short_trades)}")

            
            # Add the evaluation metrics to the DataFrame
            evaluation_df = evaluation_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Accuracy': weighted_average_accuracy,
                'Precision': weighted_average_precision,
                'Recall': weighted_average_recall,
                'F1_Score': weighted_average_f1_score,
                'Hit_Rate': weighted_average_hit_rate,
                'Long_Average_RoR': np.mean(long_avg_ror),
                'Long_Investment': np.mean(long_investment),
                'Long_Return': np.mean(long_trades),
                'Short_Average_RoR': np.mean(short_avg_ror),
                'Short_Investment': np.mean(short_investment),
                'Short_Return': np.mean(short_trades)
            }, ignore_index=True)

            # Save the model
            model.save(f"models/noWeights_combined_{timeframe}_{prediction}.h5")
            print(f"Model saved as combined_{timeframe}_{prediction}.h5")
            print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation/combined/small_evaluation_scores_combined.csv', index=False)
print("Evaluation scores saved to 'evaluation/combined/combined_model_evaluation_scores.csv'.")

Predicting 5 days ahead with 7 days timeframe.
Epoch 1/10
10/10 [==============================] - 4s 250ms/step - loss: 0.6988 - accuracy: 0.5407 - val_loss: 0.6880 - val_accuracy: 0.4935
Epoch 2/10
10/10 [==============================] - 3s 270ms/step - loss: 0.6179 - accuracy: 0.7003 - val_loss: 0.5979 - val_accuracy: 0.6883
Epoch 3/10
10/10 [==============================] - 3s 261ms/step - loss: 0.5045 - accuracy: 0.7296 - val_loss: 0.9308 - val_accuracy: 0.3896
Epoch 4/10
10/10 [==============================] - 3s 267ms/step - loss: 0.4206 - accuracy: 0.8013 - val_loss: 0.9532 - val_accuracy: 0.5065
Epoch 5/10
10/10 [==============================] - 2s 250ms/step - loss: 0.3091 - accuracy: 0.8664 - val_loss: 1.5636 - val_accuracy: 0.4416
Epoch 6/10
10/10 [==============================] - 3s 256ms/step - loss: 0.2515 - accuracy: 0.8990 - val_loss: 1.7223 - val_accuracy: 0.4416
Epoch 7/10
10/10 [==============================] - 3s 253ms/step - loss: 0.2048 - accuracy: 0.9023 -

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
10/10 [==============================] - 7s 597ms/step - loss: 0.6876 - accuracy: 0.5733 - val_loss: 0.6188 - val_accuracy: 0.7532
Epoch 2/10
10/10 [==============================] - 8s 777ms/step - loss: 0.6405 - accuracy: 0.6547 - val_loss: 0.6539 - val_accuracy: 0.7013
Epoch 3/10
10/10 [==============================] - 8s 814ms/step - loss: 0.5846 - accuracy: 0.6808 - val_loss: 0.7408 - val_accuracy: 0.4935
Epoch 4/10
10/10 [==============================] - 6s 589ms/step - loss: 0.5011 - accuracy: 0.7492 - val_loss: 0.8342 - val_accuracy: 0.4286
Epoch 5/10
10/10 [==============================] - 6s 635ms/step - loss: 0.3817 - accuracy: 0.8274 - val_loss: 1.0264 - val_accuracy: 0.4805
Epoch 6/10
10/10 [==============================] - 6s 567ms/step - loss: 0.3352 - accuracy: 0.8469 - val_loss: 1.1456 - val_accuracy: 0.5455
Epoch 7/10
10/10 [==============================] - 6s 568ms/step - loss: 0.2643 - accuracy: 0.8990 - val_loss: 1.3702 - val_accuracy: 0.5455
Epoch 

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
10/10 [==============================] - 16s 1s/step - loss: 0.7451 - accuracy: 0.5863 - val_loss: 0.7189 - val_accuracy: 0.4675
Epoch 2/10
10/10 [==============================] - 14s 1s/step - loss: 0.6689 - accuracy: 0.6124 - val_loss: 0.7042 - val_accuracy: 0.4675
Epoch 3/10
10/10 [==============================] - 14s 1s/step - loss: 0.6483 - accuracy: 0.6124 - val_loss: 0.8088 - val_accuracy: 0.4675
Epoch 4/10
10/10 [==============================] - 14s 1s/step - loss: 0.6069 - accuracy: 0.6580 - val_loss: 0.7297 - val_accuracy: 0.5065
Epoch 5/10
10/10 [==============================] - 14s 1s/step - loss: 0.5642 - accuracy: 0.7101 - val_loss: 0.8872 - val_accuracy: 0.4156
Epoch 6/10
10/10 [==============================] - 14s 1s/step - loss: 0.4830 - accuracy: 0.7752 - val_loss: 1.2101 - val_accuracy: 0.4286
Epoch 7/10
10/10 [==============================] - 14s 1s/step - loss: 0.4178 - accuracy: 0.7980 - val_loss: 1.5653 - val_accuracy: 0.4156
Epoch 8/10
10/10 [==

...

Model saved as combined_180_5.h5
--------------------------------------------------
Predicting 30 days ahead with 180 days timeframe.
Epoch 1/10
9/9 [==============================] - 77s 8s/step - loss: 0.8400 - accuracy: 0.5312 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 2/10
9/9 [==============================] - 73s 8s/step - loss: 0.6967 - accuracy: 0.4931 - val_loss: 0.6920 - val_accuracy: 0.4306
Epoch 3/10
9/9 [==============================] - 73s 8s/step - loss: 0.6762 - accuracy: 0.5764 - val_loss: 0.7093 - val_accuracy: 0.4167
Epoch 4/10
9/9 [==============================] - 75s 8s/step - loss: 0.6732 - accuracy: 0.6701 - val_loss: 0.8547 - val_accuracy: 0.3750
Epoch 5/10
9/9 [==============================] - 76s 8s/step - loss: 0.6172 - accuracy: 0.6944 - val_loss: 0.7880 - val_accuracy: 0.5417
Epoch 6/10
9/9 [==============================] - 73s 8s/step - loss: 0.5153 - accuracy: 0.7326 - val_loss: 1.1344 - val_accuracy: 0.3889
Epoch 7/10
9/9 [======================

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as combined_180_30.h5
--------------------------------------------------
Predicting 90 days ahead with 180 days timeframe.
Epoch 1/10
9/9 [==============================] - 74s 8s/step - loss: 1.1479 - accuracy: 0.5907 - val_loss: 0.6953 - val_accuracy: 0.4930
Epoch 2/10
9/9 [==============================] - 70s 8s/step - loss: 0.6437 - accuracy: 0.6868 - val_loss: 0.8064 - val_accuracy: 0.4930
Epoch 3/10
9/9 [==============================] - 70s 8s/step - loss: 0.5985 - accuracy: 0.6868 - val_loss: 0.8145 - val_accuracy: 0.4930
Epoch 4/10
9/9 [==============================] - 71s 8s/step - loss: 0.5447 - accuracy: 0.7153 - val_loss: 0.7777 - val_accuracy: 0.4930
Epoch 5/10
9/9 [==============================] - 71s 8s/step - loss: 0.5064 - accuracy: 0.7189 - val_loss: 0.7662 - val_accuracy: 0.5915
Epoch 6/10
9/9 [==============================] - 69s 8s/step - loss: 0.4671 - accuracy: 0.7687 - val_loss: 0.8481 - val_accuracy: 0.5070
Epoch 7/10
9/9 [=====================

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved as combined_180_90.h5
--------------------------------------------------
Evaluation scores saved to 'evaluation/combined/combined_model_evaluation_scores.csv'.


In [ ]:
# Get baseline RoR for each img_type, timeframe, and prediction

ror_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Average_RoR'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:

                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(img_type)) & (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)
                split_index = int(len(data) * 0.7)
                lastPrice = data['LastPrice'][split_index:]
                futurePrice = data['FuturePrice'][split_index:]
                # Calculate Rate of Return RoR
                ror = np.array((futurePrice - lastPrice)/lastPrice)
                # Calculate average RoR
                avg_ror = np.mean(ror)
                print(f"Average RoR for {img_type} {timeframe} {prediction}: {avg_ror}")
                # Add the evaluation metrics to the DataFrame
                ror_df = ror_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Average_RoR': avg_ror
                }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
ror_df.to_csv('ror.csv', index=False)

In [ ]:
# Get baseline RoR for each timeframe and prediction

ror_df = pd.DataFrame(columns=['Timeframe', 'Prediction', 'Average_RoR'])

for timeframe in TIMEFRAMES:
    for prediction in PREDICTIONS:
        if prediction < timeframe:

            # Filter your data based on prediction, img_type, and timeframe
            data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(f'_{timeframe}_'))]
            data = data.reset_index(drop=True)
            split_index = int(len(data) * 0.7)
            lastPrice = data['LastPrice'][split_index:]
            futurePrice = data['FuturePrice'][split_index:]
            # Calculate Rate of Return RoR
            ror = np.array((futurePrice - lastPrice)/lastPrice)
            # Calculate average RoR
            avg_ror = np.mean(ror)
            print(f"Average RoR for {timeframe} {prediction}: {avg_ror}")
            # Add the evaluation metrics to the DataFrame
            ror_df = ror_df.append({
                'Timeframe': timeframe,
                'Prediction': prediction,
                'Average_RoR': avg_ror
            }, ignore_index=True)

# Save the evaluation DataFrame to a CSV file
ror_df.to_csv('ror.csv', index=False)